In [20]:
import os
import pickle
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [24]:
DATA_PATH = "../data"
DICTS_PATH = "../dicts"
DATA_FILENAME = "data.csv"
data_file = os.path.join(DATA_PATH, DATA_FILENAME)
dict_60000_path = os.path.join(DICTS_PATH, "0_30000.pickle")
dict_30000_path = os.path.join(DICTS_PATH, "0_60000.pickle")
dict_90000_120000_path = os.path.join(DICTS_PATH, "0_90000_120000.pickle")
dict_120000_160000_path = os.path.join(DICTS_PATH, "0_120000_160000.pickle")
dict_160000_200000_path = os.path.join(DICTS_PATH, "0_160000_200000.pickle")
dict_200000_260000_path = os.path.join(DICTS_PATH, "0_200000_260000.pickle")
dict_260000_320000_path = os.path.join(DICTS_PATH, "0_260000_320000.pickle")
dict_320_380_path = os.path.join(DICTS_PATH, "0_320_380.pickle")
dict_380_400_path = os.path.join(DICTS_PATH, "0_380_400.pickle")

In [22]:
iter_csv = pd.read_csv(data_file, encoding="utf-16", sep='\t', iterator=True, chunksize=1000000)
df = pd.concat([chunk[(chunk['Закупки по'] == '44-ФЗ') 
                & (chunk['Этап закупки']=='Определение поставщика завершено')
                & (chunk["Дата обновления"].str.split('.', expand=True).iloc[:, 2].astype("int").isin([2019, 2020, 2021]))] 
                for chunk in tqdm(iter_csv)])

11it [04:47, 26.17s/it]


In [28]:
df.shape

(2152814, 26)

In [25]:
with open(dict_90000_120000_path, 'rb') as handle:
    d_90_120 = pickle.load(handle)
with open(dict_30000_path, 'rb') as handle:
    d_30 = pickle.load(handle)
with open(dict_60000_path, 'rb') as handle:
    d_60 = pickle.load(handle)
with open(dict_120000_160000_path, 'rb') as handle:
    d_120_160 = pickle.load(handle)
with open(dict_160000_200000_path, 'rb') as handle:
    d_160_200 = pickle.load(handle)
with open(dict_200000_260000_path, 'rb') as handle:
    d_200_260 = pickle.load(handle)
with open(dict_260000_320000_path, 'rb') as handle:
    d_260_320 = pickle.load(handle)
with open(dict_320_380_path, 'rb') as handle:
    d_320_380 = pickle.load(handle)
with open(dict_380_400_path, 'rb') as handle:
    d_380_400 = pickle.load(handle)

In [31]:
def convert(p):
    if p in d_30:
        return d_30[p]
    else:
        if p in d_60:
            return d_60[p]
        else:
            if p in d_90_120:
                return d_90_120[p]
            else:
                if p in d_120_160:
                    return d_120_160[p]
                else:
                    if p in d_160_200:
                        return d_160_200[p]
                    else:
                        if p in d_200_260:
                            return d_200_260[p]
                        else:
                            if p in d_260_320:
                                return d_260_320[p]
                            else:
                                if p in d_320_380:
                                    return d_320_380[p]
                                else:
                                    if p in d_380_400:
                                        return d_380_400[p]
                                    else: return

In [32]:
df['purch'] = df['Реестровый номер закупки'].apply(lambda x: x[1:])
df["winer"] = df["purch"].apply(lambda p: convert(p))
df = df[df.winer != 0]
inns = df.groupby("winer").winer.count()[df.groupby("winer").winer.count() > 15].index
df_val = df[df["winer"].isin(inns)]

In [33]:
df_val["Дата обновления"] = pd.to_datetime(df_val["Дата обновления"], format="%d.%m.%Y")
df_val["Год обновления"] = df_val["Дата обновления"].apply(lambda date: date.year)

In [34]:
stat = df_val.groupby(["winer", "Год обновления"]).winer.count()

In [35]:
valid_inns = []
for inn in inns:
    inn_stat = stat[inn]
    if inn_stat.shape[0] != 3:
        continue
    f = True
    for year in inn_stat.index[1:]:
        if (inn_stat[year] / inn_stat[year - 1]) < 0.75:
            f = False
            break
    if f: valid_inns.append(inn)

In [36]:
len(valid_inns)

541

In [37]:
df_val = df_val[df_val.winer.isin(valid_inns)]

In [38]:
df_val.shape

(23768, 28)

In [39]:
df_val.to_csv(os.path.join(DATA_PATH, "valid.csv"), encoding="utf-8", sep=',')

In [40]:
count_2019 = lambda x: x[x == 2019].count()
count_2019.__name__ = "Кол-во (2019)"
count_2020 = lambda x: x[x == 2020].count()
count_2020.__name__ = "Кол-во (2020)"
count_2021 = lambda x: x[x == 2021].count()
count_2021.__name__ = "Кол-во (2021)"

# stat = df_val.groupby(["winer"]).agg({
#     "Год обновления": lambda x: (x == 2019).sum(),
#     "Год обновления": lambda x: (x == 2020).sum()})

stat = df_val.groupby(["winer"]).agg(
    count_2019=("Год обновления", lambda x: (x == 2019).sum()),
    count_2020=("Год обновления", lambda x: (x == 2020).sum()),
    count_2021=("Год обновления", lambda x: (x == 2021).sum()),
    amount_2019=("Начальная (максимальная) цена контракта", lambda x: x.sum())
)

stat

,count_2019,count_2020,count_2021,amount_2019
winer,,,,
0224013045,46,54,46,8.320833e+07
0269998410,4,8,10,1.142172e+07
0273041550,11,15,17,3.052835e+06
0274110679,31,52,57,2.468903e+10
0275900969,7,8,12,7.056130e+07
...,...,...,...,...
9715229846,1,9,7,1.349851e+07
9715322330,6,10,14,2.247411e+07
9715338098,1,9,22,5.874991e+07


In [41]:
count_df = df_val.pivot_table("Закупки по", index="winer", columns=["Год обновления"], aggfunc='count')
count_df.rename(columns={2019: "2019 кол-во", 2020: "2020 кол-во", 2021: "2021 кол-во"}, inplace = True)
count_df

Год обновления,2019 кол-во,2020 кол-во,2021 кол-во
winer,,,
0224013045,46,54,46
0269998410,4,8,10
0273041550,11,15,17
0274110679,31,52,57
0275900969,7,8,12
...,...,...,...
9715229846,1,9,7
9715322330,6,10,14
9715338098,1,9,22


In [42]:
sum_df = df_val.pivot_table("Начальная (максимальная) цена контракта", index="winer", columns=["Год обновления"], aggfunc='sum')
sum_df.rename(columns={2019: "2019 сумма", 2020: "2020 сумма", 2021: "2021 сумма"}, inplace = True)
sum_df

Год обновления,2019 сумма,2020 сумма,2021 сумма
winer,,,
0224013045,1.978899e+07,4.207305e+07,2.134629e+07
0269998410,7.493098e+05,3.165213e+06,7.507196e+06
0273041550,6.173826e+05,1.242729e+06,1.192723e+06
0274110679,1.663205e+09,2.146573e+10,1.560091e+09
0275900969,7.099961e+06,2.308415e+07,4.037719e+07
...,...,...,...
9715229846,1.751351e+06,5.176178e+06,6.570980e+06
9715322330,1.516988e+06,8.566969e+06,1.239015e+07
9715338098,2.313520e+06,2.561772e+07,3.081867e+07


In [43]:
pd.concat((count_df, sum_df), axis=1).to_csv("../data/info.csv")